In [1]:
import json
import os
import geopandas as gpd
from glob import glob
from shapely.geometry import box
from shapely.geometry import Polygon
from datetime import date
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
def read_json(path_file):
    with open(path_file, 'r') as file:
        data = json.load(file)
    return data

In [3]:
def write_json(id_wilayah, dict_list):
    path_file = f'/data/ksa/01_Image_Acquisition/05_Json_Coverage/{id_wilayah}_coverage_ASF.json'
    with open(path_file, 'w') as json_file:
        json.dump(dict_list, json_file, ensure_ascii=False, indent=4)

In [4]:
def get_df_asf(path_file):
    data = read_json(path_file)

    asf_images = []
    for item in data['features']:
        filename = item['properties']['fileName'][:-4]
        tahun = item['properties']['processingDate'][:4]
        bulan = item['properties']['processingDate'][5:7]
        tanggal = item['properties']['processingDate'][8:10]
        geometry = item['geometry']['coordinates'][0]
        polygon = Polygon(geometry)
        asf_images.append(
           {
               'filename': filename, 
               'tahun': tahun,
               'bulan': bulan,
               'tanggal': tanggal,
               'geometry': polygon
           }
        )
    return gpd.GeoDataFrame(asf_images, geometry='geometry')

In [5]:
def get_df_input(id_wilayah):
    gdf_mgrs = gpd.read_file('/data/ksa/00_Data_Input/indo_mgrs_grid_100km_overlay.gpkg')
    
    if len(id_wilayah) == 2 :
        gdf_wilayah = gpd.read_file('/data/ksa/00_Data_Input/provinsi.gpkg')
        gdf_wilayah = gdf_wilayah[gdf_wilayah['provno'] == id_wilayah]
    else :
        gdf_wilayah = gpd.read_file('/data/ksa/00_Data_Input/kabupaten.gpkg')
        gdf_wilayah = gdf_wilayah[gdf_wilayah['idkab'] == id_wilayah]
        
    intersecting_gdf = gpd.sjoin(gdf_mgrs, gdf_wilayah, how='inner', predicate='intersects')
    gdf_mgrs = gdf_mgrs[gdf_mgrs.index.isin(intersecting_gdf.index)]
    return gdf_mgrs, gdf_wilayah

In [49]:
def generate_date_pairs(year):
    start_date = datetime(year, 1, 1)
    date_pairs = []
    
    while start_date.year == year:
        end_date = start_date + timedelta(days=11)
        # Ensure the end date is still within the same year
        if end_date.year != year:
            break

        start_date_str = "2023-08-01"
        end_date_str = "2024-09-30"
        
        start_periode = datetime.strptime(start_date_str, "%Y-%m-%d")
        end_periode = datetime.strptime(end_date_str, "%Y-%m-%d")
        
        # Check if the date falls within the range
        if start_periode <= start_date <= end_periode:
            date_pairs.append([start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')])
        start_date = end_date + timedelta(days=1)
    return date_pairs

In [7]:
from tqdm import tqdm

def get_periode_data(row, periode, gdf_asf):
    startPeriode = periode[0].split('-')
    endPeriode = periode[1].split('-')

    periode = {
        'start_periode': periode[0],
        'end_periode': periode[1]
    }

    gdf_asf_filter = gdf_asf[
        ((gdf_asf['tahun'] >= startPeriode[0]) & (gdf_asf['bulan'] >= startPeriode[1]) & (gdf_asf['tanggal'] >= startPeriode[2])) &
        ((gdf_asf['tahun'] <= endPeriode[0]) & (gdf_asf['bulan'] <= endPeriode[1]) & (gdf_asf['tanggal'] <= endPeriode[2]))
    ]
    
    if gdf_asf_filter.empty:
        periode.update({
            'image_asf': '',
            'coverage': 0,
            'alasan': 'Tidak ada image periode tersebut'
        })
        return periode

    overlapping_rows = gdf_asf_filter[gdf_asf_filter.intersects(row['geometry'])]
    if overlapping_rows.empty:
        periode.update({
            'image_asf': '',
            'coverage': 0,
            'alasan': 'Tidak ada image yang overlap dengan MGRS'
        })
        return periode

    periode['image_asf'] = overlapping_rows['filename'].to_list()
    df_temp = overlapping_rows.dissolve()
    area1 = row['geometry'].area
    intersection = row['geometry'].intersection(df_temp['geometry'])[0].area
    percentage_covered = (intersection / area1) * 100
    periode['coverage'] = percentage_covered

    periode['alasan'] = 'Coverage di bawah 80%' if percentage_covered < 80 else 'Sudah OK!'
    return periode

In [36]:
def process_rows(gdf_mgrs, gdf_asf):
    dict_list = []
    periode_range = []
    years = [2023, 2024]
    for year in years:
        date_pairs = generate_date_pairs(year)
        date_pairs[-1][1] = date_pairs[-1][1][:-2] + '31'
        periode_range = periode_range + date_pairs
    
    for _, row in tqdm(gdf_mgrs.iterrows(), total=gdf_mgrs.shape[0]):
        periode_list = [get_periode_data(row, periode, gdf_asf) for periode in periode_range]
        
        dict_list.append({
            'id': row['MGRS'],
            'geometry': row['geometry'].wkt,
            'periode': periode_list
        })
    
    return dict_list

### Running For Kabupaten

In [29]:
list_file = glob('/data/ksa/01_Image_Acquisition/04_Json_Raw_Download/*_groundtruth_metadata_ASF.json')

In [50]:
for path_file in list_file:
    id_wilayah = path_file.split('/')[-1][:4]
    gdf_mgrs, gdf_wilayah = get_df_input(id_wilayah)
    gdf_asf = get_df_asf(path_file)
    dict_list = process_rows(gdf_mgrs, gdf_asf)
    write_json(id_wilayah, dict_list)

100%|██████████| 3/3 [00:00<00:00,  5.26it/s]


In [22]:
list_file = sorted(glob('/data/ksa/01_Image_Acquisition/05_Json_Coverage/*_coverage_ASF.json'))

In [25]:
for i in list_file:
    id_wilayah = i.split('/')[-1].split('_')[0]
    if len(id_wilayah) == 4: 
        data = read_json(i)
        
        break

### Running for Provinsi

In [ ]:
id_wilayah = '32'

In [ ]:
gdf_mgrs, gdf_wilayah = get_df_input(id_wilayah)
gdf_asf = get_df_asf(id_wilayah)

In [ ]:
dict_list = process_rows(gdf_mgrs, gdf_asf)

In [ ]:
write_json(id_wilayah, dict_list)

In [ ]:
from IPython.display import display, Markdown
markdown_link = f"### Lihat Result [Disini](../../01_Image_Acquisition/05_Json_Coverage/{id_wilayah}_coverage_ASF.json)"
display(Markdown(markdown_link))